In [7]:

%%capture --no-stderr
%pip install --upgrade --quiet langchain-community langgraph
%pip install -qU "langchain[openai]"
%pip install -U langchain-anthropic



In [1]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/nikhilw/Documents/Devel/PyProj_SQLAgent


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinoook.db")
print(db.dialect)
print(db.get_usable_table_names())
# db.run("SELECT name FROM sqlite_master LIMIT 10;")
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [3]:
db.get_usable_table_names()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [4]:
""" import getpass
import os

if not os.environ.get("ANTHROPIC_API_KEY"):
  os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter API key for Anthropic: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")

try:
	response = llm.invoke("What is the capital of France?")
	print("Response:", response)
except Exception as e:
	print("Error connecting to Anthropic API:", e)
	print("Please check your internet connection, firewall settings, and API key.")
print("llm:", llm)
 """

' import getpass\nimport os\n\nif not os.environ.get("ANTHROPIC_API_KEY"):\n  os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter API key for Anthropic: ")\n\nfrom langchain.chat_models import init_chat_model\n\nllm = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")\n\ntry:\n\tresponse = llm.invoke("What is the capital of France?")\n\tprint("Response:", response)\nexcept Exception as e:\n\tprint("Error connecting to Anthropic API:", e)\n\tprint("Please check your internet connection, firewall settings, and API key.")\nprint("llm:", llm)\n '

In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
	os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

try:
	response = llm.invoke("What is the capital of India?")
	print("Response:", response)
except Exception as e:
	print("Error connecting to OPENAI API:", e)
	print("Please check your internet connection, firewall settings, and API key.")
print("llm:", llm)


Response: content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be6jw2YvWjgmjMKnwkKZ2Pj0X9HCz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--901cbf94-91b2-4888-8d2e-6b6d8e18934c-0' usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
llm: client=<openai.resources.chat.completions.completions.Completions object at 0x10ba29150> async_client=<openai.resources.chat.completions.completions.AsyncCompletions ob

In [6]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str # User's question to be answered.
    query: str # SQL query generated from the question.
    result: str # Result of the query execution in json.
    answer: str  # Answer to the question based on the query result.
    columns: list[str]  # List of column names in the result.
    followup: list[str] # List of followup questions 

In [7]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
Output only 1 query even for multiple questions. 
All decimal numbers in the output should be formatted to only 2 digits after the decimal.
Do not add any explanation. Do not add any comments.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
Output only 1 query even for multiple questions. 
All decimal numbers in the output should be formatted to only 2 digits after the decimal.
Do not add any explanatio

In [8]:
db.get_table_info(["Artist", "Album", "Track", "Customer", "Invoice", "InvoiceLine", "Genre", "MediaType"])

'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

In [9]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    print("Prompt:", prompt)
    # print("Prompt:", prompt.messages[1].content)

    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [10]:
 write_query({"question": "Whats the Total Invoice for each month?"})

Prompt: messages=[SystemMessage(content='\nGiven an input question, create a syntactically correct sqlite query to\nrun to help find the answer. Unless the user specifies in his question a\nspecific number of examples they wish to obtain, always limit your query to\nat most 10 results. You can order the results by a relevant column to\nreturn the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the\nfew relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema\ndescription. Be careful to not query for columns that do not exist. Also,\npay attention to which column is in which table.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\nOutput only 1 query even for multiple questions. \nAll decimal numbers in the output should be formatted to only 2 digits after the decimal.\nDo not add any explanation. Do not add any comments.\n\nO

{'query': "SELECT strftime('%Y-%m', InvoiceDate) AS Month, SUM(Total) AS TotalInvoice\nFROM Invoice\nGROUP BY Month\nORDER BY Month\nLIMIT 10;"}

In [11]:
from typing_extensions import Annotated

class ColumnOutput(TypedDict):
    columns: Annotated[list[str], "List of column names in the SELECT statement, in order."]

def extract_columns(state: State):
    """Use LLM to extract column names from the SQL query."""
    prompt = (
        "Given the following SQL query, list the column names that will appear in the result set, in order, as a Python list of strings.\n\n"
        f"SQL Query:\n{state['query']}\n\n"
        "Columns:"
    )
    structured_llm = llm.with_structured_output(ColumnOutput)
    result = structured_llm.invoke(prompt)
    return {"columns": result["columns"]}

In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    str_result = execute_query_tool.invoke(state["query"])
    # print("Query:", state["query"])
    return {"result": str_result}

In [13]:
execute_query({"query": "SELECT COUNT(*) AS TotalEmployees FROM Employee UNION SELECT COUNT(*) AS TotalArtists FROM Artist;"})

{'result': '[(8,), (275,)]'}

In [14]:
from typing_extensions import Annotated

class QuestionList(TypedDict):
    questions: Annotated[list[str], "List of followup questions."]

def FollowupQuestions(state: State):
    """Use LLM to extract column names from the SQL query."""
    prompt = (
        "Given the following SQL query and Result, list the next 3 followup questions that might be relevant for the user.\n\n"
        f"SQL Query:\n{state['query']}\n\n"
        f"Result:\n{state['result']}\n\n"
        "Follow-up Questions:"
    )
    structured_llm = llm.with_structured_output(QuestionList)
    print("Prompt:", prompt)
    result = structured_llm.invoke(prompt)
    return {"Followup_Questions": result["questions"]}

In [15]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'

    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [16]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, extract_columns, execute_query, generate_answer, FollowupQuestions]  # Insert extract_columns after write_query
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [20]:
str_query = ""
str_result = ""
str_answer = ""
str_columns = []
for step in graph.stream(
    {"question": "Reset total revenue for Aerosmith to 100.00"}, stream_mode="updates"
#    {"question": "Can we see a breakdown of Aerosmith's revenue by album or track for each of those years to identify their top-performing content? ?"}, stream_mode="updates"
):
    print("Step:", step)
    if "write_query" in step:
        str_query = step["write_query"]["query"]
    if "execute_query" in step:
        str_result = step["execute_query"]["result"]
    if "generate_answer" in step:
        str_answer = step["generate_answer"]["answer"]
    if "extract_columns" in step:
        str_columns = step["extract_columns"]["columns"]
    if "Followup_Questions" in step:
        followup_questions = step["FollowupQuestions"]["Followup_Questions"]
    print("Query :", str_query)





Prompt: messages=[SystemMessage(content='\nGiven an input question, create a syntactically correct sqlite query to\nrun to help find the answer. Unless the user specifies in his question a\nspecific number of examples they wish to obtain, always limit your query to\nat most 10 results. You can order the results by a relevant column to\nreturn the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the\nfew relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema\ndescription. Be careful to not query for columns that do not exist. Also,\npay attention to which column is in which table.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\nOutput only 1 query even for multiple questions. \nAll decimal numbers in the output should be formatted to only 2 digits after the decimal.\nDo not add any explanation. Do not add any comments.\n\nO

In [18]:
from IPython.display import display, HTML
import ast

# Convert the string representation of the list to an actual list
result_list = ast.literal_eval(str_result)

# Build HTML table
html = "<table><tr><th>Artist.Name</th><th>year</th><th>TotalRevenue</th></tr>"
for name, year, total in result_list:
    html += f"<tr><td>{name}</td><td>{year}</td><td>{total}</td></tr>"
html += "</table>"

display(HTML(html))

ValueError: not enough values to unpack (expected 3, got 2)